In [ ]:
!pip install pandas scikit-learn torch transformers imbalanced-learn nlpaug

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from torch.utils.data import Dataset
from sklearn.metrics import classification_report
import csv

# 데이터 전처리

In [ ]:
# CSV 데이터 로드
df = pd.read_csv('final.csv', quoting=csv.QUOTE_NONE, on_bad_lines='skip')

In [ ]:
# 댓글 열을 문자열로 변환하고 결측값 제거
df['comments'] = df['comments'].astype(str)
df.dropna(subset=['comments', 'type'], inplace=True)

In [ ]:
# 클래스 1 데이터 제거
df = df[df['type'] != 1]

In [ ]:
# 클래스 라벨을 0과 1로 변환
df['type'] = df['type'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
# 데이터셋을 훈련과 테스트로 나누기
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# 훈련 데이터와 라벨 분리
train_comments = train_df[['comments']]
train_labels = train_df['type']

In [ ]:
# 원본 훈련 데이터 분포 확인
print("Original training set distribution:")
print(train_df['type'].value_counts())

# 데이터셋 클래스 정의

In [ ]:
# 데이터셋 클래스 정의
class CommentDataset(Dataset):
    def __init__(self, comments, labels, tokenizer, max_length):
        self.comments = comments
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, idx):
        comment = self.comments[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            comment,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

# 토크나이저 및 모델 로드

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

In [ ]:
# 훈련 데이터셋 생성
train_dataset = CommentDataset(
    comments=train_comments['comments'].tolist(),
    labels=train_labels.tolist(),
    tokenizer=tokenizer,
    max_length=128
)

In [ ]:
# 테스트 데이터셋 생성
test_dataset = CommentDataset(
    comments=test_df['comments'].tolist(),
    labels=test_df['type'].tolist(),
    tokenizer=tokenizer,
    max_length=128
)

In [ ]:
# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base", num_labels=2)

# TraingArguments 및 Trainer 설정

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='saved_model/hazard/results',
    num_train_epochs=30,
    per_device_train_batch_size=16,  # 배치 크기를 16으로 증가시킴
    per_device_eval_batch_size=16,   # 평가 배치 크기도 동일하게 증가
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='saved_model/hazard/logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    fp16=True,  # mixed precision training 활성화
    save_strategy="epoch",  # 각 에포크마다 모델을 저장
    eval_strategy="epoch",
    load_best_model_at_end=True,  # 최적의 모델을 마지막에 로드
    save_total_limit=3
)

In [ ]:
# Trainer 생성
trainer = Trainer(
    model=model,  # 모델
    args=training_args,  # 훈련 인자
    train_dataset=train_dataset,  # 훈련 데이터셋
    eval_dataset=test_dataset,  # 평가 데이터셋
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# 모델 훈련

In [ ]:
# 모델 훈련
trainer.train()

# 모델 평가 및 성능 평가

In [ ]:
# 모델 평가
results = trainer.evaluate()
print(results)

In [ ]:
# 성능 평가
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
labels = predictions.label_ids
print(classification_report(labels, preds))

In [ ]:
# 모델과 토크나이저 저장
model.save_pretrained('saved_model/hazard')
tokenizer.save_pretrained('saved_model/hazard')

print("Model and tokenizer saved to './saved_model/hazard'")